In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/moby_dick.txt

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import random
import string
import re

In [ ]:
# Helper functions
def has_duplicates(array):
    return len(array) != len(set(array))

def format_text(text):    
    # Remove punctuation and digits
    formatted_text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # remove more than one space in a row
    formatted_text = re.sub(r'\s{2,}', ' ', formatted_text)
    
    # Convert to lowercase
    formatted_text = formatted_text.lower()
    # Remove spaces
    #formatted_text = upper_text.replace(' ', '')
    
    return formatted_text


def decode(array,dict):
    #array = format_text(array)
    decoded = []
    for x in array:
        try:
            decoded.append(dict[x])
        except:
            decoded.append(" ")
    
    return decoded

def encode(text,cypher):
    text = format_text(text)
    
    encoded = []
    for x in text:
        try:
            encoded.append(cypher[x])
        except:
            encoded.append(" ")
        
    return encoded




In [ ]:
# initialize markov model
M = np.ones((26,26))

# initial state distribution
pi = np.zeros(26)

# function to update Markov model
def update_M(char1, char2 ):
    # convert letters to ascii to place them 
    # in the correct place in the Matrix M
    # 'A' = 97 , 'B' = 98 ...
    i = ord(char1) - 97
    j = ord(char2) - 97
    M[i,j] +=1
    
def update_init_distr(char1):
    i = ord(char1) - 97
    if i > 26:
        print(i)
    pi[i] += 1
    
def get_log_prob(word):
    i = ord(word[0]) - 97
    if i < 0:
        return 0
    logp = np.log(pi[i])
    
    for ch in word[1:]:
        j = ord(ch) - 97
        logp += np.log(M[i,j])
       
        i = j
    
    return logp

# get probability of a sequence of words
def get_sentence_log_prob(text):
    if type(text) == str:
        text = text.split()
        
    logp = 0
    for word in text:
        logp += get_log_prob(word)
    
    return logp
    
    
    

In [ ]:
    

for line in open('moby_dick.txt' , encoding='utf-8'):
    line = line.rstrip()
    
    if line:
        line = format_text(line)
        
        tokens = line.split()
        
        for token in tokens:
            ch0 = token[0]
            update_init_distr(ch0)
            
            for ch1 in token[1:]:
                update_M(ch0,ch1)
                ch0 = ch1
    
    pi /= pi.sum()
    M /= M.sum(axis=1,keepdims=True)
    
    
    

In [ ]:

# implement evolutionary algorithm to decode the message
alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
alphabet = [x.lower() for x in alphabet]


enc_cypher = alphabet[:]
random.seed(1234)
random.shuffle(enc_cypher)

enc_dict = dict(zip(alphabet, enc_cypher))


message = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

enc_message = encode(message,enc_dict)

dna_pool = []
for i in range(20):
    dna = alphabet[:]
    random.shuffle(dna)
    dna_pool.append(dna)

In [ ]:
def evolve_offspring(dna_pool,n_children):
    # make n children for each offspring
    offspring = []
    
    for dna in dna_pool:    
        for _ in range(n_children):
            copy = dna.copy()
            # make random swaps 
            j,k = random.sample(range(len(copy)),2)
            copy[j], copy[k] = copy[k],copy[j]
             
            offspring.append(copy)
    
    return offspring + dna_pool


num_iters = 1000
scores = np.zeros(num_iters)
best_dna = None
best_map = None
best_score = float('-inf')

for i in range(num_iters):
    if i>0:
        dna_pool = evolve_offspring(dna_pool,3)
        
    dna2score = {}
    for dna in dna_pool:
        current_map = {}
        for k,v in zip(alphabet,dna):
            current_map[k] = v
        
        decoded_message = decode(enc_message,current_map)
        score  = get_sentence_log_prob(decoded_message)
        
        dna2score[''.join(dna)] = score
        
        if score > best_score:
            best_dna = dna
            best_map = current_map
            best_score = score
            
    scores[i] = np.mean(list(dna2score.values()))
    
    # keep 5 best dna
    sorted_dna = sorted(dna2score.items(),key= lambda x:x[1], reverse= True)
    dna_pool = [list(k) for k,v in sorted_dna[:5]]
    
    if i % 200 == 0:
        print("iter:", i , "score:", scores[i], "best so far", best_score)        
        




In [ ]:
import matplotlib.pyplot as plt
plt.plot(scores)
plt.show()